In [19]:
import pandas as pd
import csv

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import cross_val_score

from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import sys

#import re # Регулярные выражения.
#import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.

/Applications/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df_sample = pd.read_csv ('samplesubmission.csv', sep = '\t')
df_train = pd.read_csv ('train.csv', sep = '\t')
df_other = pd.read_csv ('other.csv', sep = '\t')
df_test = pd.read_csv ('test.csv', sep = '\t')

y = df_train['target'].values
y_t = df_train.loc[:100,['target']].values

df_train = df_train.drop(['target'], axis=1)
t = df_train.loc[:100,:]

In [8]:
def show_progress(value):

    sys.stderr.write('%d\r' % value)


    
def preproc_data(df_input):
    df_output = df_input.copy()

    #удаляем теги и пихаем в один столбец
    for i in range(len(df_output['name'])):
        df_output.loc[i, 'name_description'] = df_output.loc[i, 'name'] + " " + BeautifulSoup(df_output.loc[i, 'description']).get_text()
        show_progress(i)

    df_output = df_output.drop(['id', 'name', 'description'], axis=1)
    
    return df_output

In [13]:
X = preproc_data(df_train)

/Applications/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [14]:
model = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1,1),
                             max_df=0.95, min_df=5)),
    ('clf', LogisticRegression(random_state = 123)),
])


model2 = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1,1))),
    ('clf', LogisticRegression(random_state = 123)),
])


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X['name_description'].values, y, test_size=0.2)

In [18]:
model.fit(X_train, y_train)
model2.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...lty='l2', random_state=123, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [22]:
scores = cross_val_score(
    model,
    X_train,
    y_train,
    scoring='roc_auc',
    cv=5
)
##print('Лог регрессия')
print(scores)
print(scores.mean())

scores2 = cross_val_score(
    model2,
    X_train,
    y_train,
    scoring='roc_auc',
    cv=5
)
##print('Лог регрессия')
print(scores2)
print(scores2.mean())

[0.99021936 0.99103473 0.99095972 0.98999496 0.99009998]
0.9904617501312154
[0.98859245 0.98928438 0.98942545 0.98911314 0.98950796]
0.9891846751727961


In [21]:
print(len(df_test['name']))

170179


In [23]:
df = preproc_data(df_test)

/Applications/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Applications/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [37]:
#сразу делаем на тестовой


y_hat = model.predict_proba(df['name_description'].values)[:,1]

X_test = df['name_description'].values
y_test = pd.Series(y_hat)

In [38]:
res = pd.DataFrame(df_test['id'])
##res['Target'] = pd.Series(y_hat).round(decimals=0).astype(int)
res['Target'] = pd.Series(y_hat)
res['Target'].value_counts()
res

,id,Target
0,200000,0.979575
1,200001,0.998910
2,200002,0.995551
3,200003,0.995552
4,200004,0.007085
5,200005,0.060765
6,200006,0.994635
7,200007,0.073372
8,200008,0.031837
9,200009,0.006795


In [39]:
res.to_csv('submission_2.csv', sep = ',', index=False)
